In [12]:
import torch.nn as nn
import torch.optim as optim
from classes import SpeedEstimatorRNN, VehicleSpeedDataset
from torch.utils.data import DataLoader
import torch

In [13]:
if torch.cuda.is_available():
    print("CUDA is available! You can use a GPU for training.")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU being used:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Training will be performed on the CPU.")

CUDA is available! You can use a GPU for training.
Number of GPUs available: 1
Current GPU being used: 0
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [16]:
# Set dataset path
data_path = r'C:\my files\thesis\AI_training\data\i7\it_1\1_training'
extension = "*.csv"

# Hyperparameters that will alter throughout the model creations
input_size = 20  # Number of CAN signals per timestep
hidden_size = [256, 256, 512, 512, 512]
num_layers = [3, 4, 3, 4, 3]
learning_rate = [0.0001] * 5
# num of sequences in one batch
batch_size = [128] * 5
dropout_rate = [0.2] * 5
sequence_length = [800, 800, 800, 800, 1000]


# parameters of the simulation
step_size = 10 # what the overlap between the sequences should look like in the extracted dataset
output_size = 2
num_epochs = 20

num_models = 5

default_location = r"C:\my files\thesis\AI_training\Simple RNN\trained_models\iteration_8\model_"

In [5]:
dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[1], step_size = step_size)

56


In [ ]:
# Training loops
for j in range(num_models):
    # Load dataset and DataLoader
    dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[j], step_size = step_size)
    dataloader = DataLoader(dataset, batch_size=batch_size[j], shuffle=True, num_workers= 6, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SpeedEstimatorRNN(input_size, hidden_size[j], num_layers[j], output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[j])

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx, (features, speeds) in enumerate(dataloader):
            speeds = speeds.squeeze(1)  # Remove extra dimension from speeds if present
            features, speeds = features.to(device), speeds.to(device)

            # Forward pass
            outputs = model(features)

            assert outputs.shape == speeds.shape, f"Shape mismatch: outputs {outputs.shape} vs speeds {speeds.shape}"

            loss = criterion(outputs, speeds)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Model: {j}, Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "sequence_length": sequence_length,
        "input_size": input_size,
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "output_size": output_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs
    }, default_location + str(j) + ".pt")